In [1]:
import sys
sys.path.insert(0, './cube_2025')
from cube import Cube
from solver import Solver
from visualize import print_color_cube
import json

In [2]:
cube = Cube()
solver = Solver(cube)
cube.scramble(20)
solver.cross()

print_color_cube(cube)


K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [3]:
solver.f2l()
print_color_cube(cube)
print(str(cube))
def get_matching_edge(corner):
        """
        Get the edge that belongs to the corner cubie
        """
        color_filter = ''.join(corner.color).replace('W', '')
        edges = corner.cube.edges(color_filter=color_filter)
        for edge in edges:
            matched = True
            for color in edge.color:
                if color not in color_filter:
                    matched = False
            if matched:
                return edge
        return None


K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

WWWWWWWWWYYOYYRBOGGGGGGGYGRBBBBBBYYRRRRRRRBBGOOOOOOYYO


In [4]:
with open("f2l_sequence_data.json", "r") as f:
    f2l_data = json.load(f)
edge_sequences = f2l_data["edge_sequences"]
corner_sequences = f2l_data["corner_sequences"]

def f2l_place_corner_edge(cube, corner, edge):
    if 'D' in corner.position:
        print(f"Corner {corner} is in the D layer")
        while corner.position != 'DFR':
            print(f"Corner {corner} is not in the DFR position, rotating cube")
            cube.rotate_cube(axis='Y', clockwise=True)
        print(f"Corner {corner} is now in the DFR position")
        if corner.alignment() == 3:
            if cube.get_sticker('D', cubie=corner) == cube.face_color('D'):
                print(f"Corner is in the correct position: {corner}")
                if 'U' in edge.position:
                    while cube.face_color(edge.position[1]) not in edge.color:
                        cube.rotate_face('U', clockwise=True)
                    condition = cube.get_sticker('U', cubie=edge) == cube.face_color('F')
                elif 'F' in edge.position:
                    condition = cube.get_sticker('F', cubie=edge) == cube.face_color('F')
                else:
                    condition = None
                if edge.position in corner_sequences['D']['W']:
                    if condition in corner_sequences['D']['W'][edge.position]:
                        cube.sequence(corner_sequences['D']['W'][edge.position][condition])
                    else:
                        return True
            else:
                print(f"Sending corner {corner} to the top 1")
                cube.sequence("R,U,R'")
        else:
            print(f"Corner {corner}, alignment: {corner.alignment()} is in the wrong position")
            cube.sequence("R,U,R'")
    else:
        # move middle edge to the top
        print(f"Corner pre_alignment: {corner}, {corner.alignment()}")
        while corner.alignment() != 2:
            cube.rotate_face(face='U', clockwise=True)
            print(f"Corner U: {corner}, {corner.alignment()}")
        print(f"Corner post_alignment: {corner}, {corner.alignment()}")
        while corner.position != 'UFR':
            cube.rotate_cube(axis='Y', clockwise=True)
        print(f"Corner {corner} is now in the UFR position")
        print(f"edge.position in edge_sequences: {edge.position} {edge.position in edge_sequences}")
        if edge.position in edge_sequences:
            cube.sequence(edge_sequences[edge.position])
        print(f"Edge {edge} ")
        right_color = cube.get_sticker('R', 4)
        front_color = cube.get_sticker('F', 4)
        corner_up_color = cube.get_sticker('U', cubie=corner)
        corner_condition = str(front_color == corner_up_color).lower()
        edge_up_color = None
        if 'U'in edge.position:
            edge_up_color = cube.get_sticker('U', cubie=edge)
            edge_condition = str(edge_up_color == front_color).lower()
        else:
            edge_condition = str(cube.get_sticker('F', cubie=edge) == front_color).lower()
        
        print(f"Corner condition: {corner_condition}, Edge condition: {edge_condition}")
        sequence = None
        if corner_up_color in corner_sequences['U']:
            sequence = corner_sequences['U'][corner_up_color].get(edge.position, {}).get(edge_condition, None)
            print(f"U {corner_up_color} {edge.position} {edge_condition} sequence: {sequence}")
        else:
            sequence = corner_sequences['U'].get(corner_condition, {}).get(edge.position, {}).get(edge_condition, None)
            print(f"U {corner_condition} {edge.position} {edge_condition} sequence: {sequence}")
        if sequence is not None:
            print(f"Executing sequence for corner {corner} and edge {edge}: {sequence}")
            cube.sequence(sequence)
            # print_color_cube(cube)
    return False
            



        


In [5]:
white_corners = cube.corners(color_filter=["W"])
print(f"White corners: {white_corners}")

for _ in range(5):
    for corner in white_corners:
        print()
        print(f"Corner {corner} alignment: {corner.alignment()}")
        edge = get_matching_edge(corner)
        print(f"Matching edge for corner {corner}: {edge}")
        f2l_place_corner_edge(cube, corner, edge)

print_color_cube(cube)


White corners: [Cubie(color=('W', 'R', 'G'), position=UFL, orientation=0), Cubie(color=('W', 'B', 'R'), position=UFR, orientation=0), Cubie(color=('W', 'O', 'B'), position=UBR, orientation=0), Cubie(color=('W', 'G', 'O'), position=UBL, orientation=0)]

Corner ('W', 'R', 'G') UFL 0 alignment: 3
Matching edge for corner ('W', 'R', 'G') UFL 0: ('R', 'G') FL 0
Corner pre_alignment: ('W', 'R', 'G') UFL 0, 3
Corner U: ('W', 'R', 'G') UBL 0, 2
Corner post_alignment: ('W', 'R', 'G') UBL 0, 2
Corner ('W', 'R', 'G') UFR 0 is now in the UFR position
edge.position in edge_sequences: BR True
Edge ('R', 'G') UR 0 
Corner condition: false, Edge condition: false
U W UR false sequence: Y',R',U',R,U,U,R',U',R,U,R',U',R
Executing sequence for corner ('W', 'R', 'G') UBR 0 and edge ('R', 'G') UR 0: Y',R',U',R,U,U,R',U',R,U,R',U',R

Corner ('W', 'B', 'R') DBR 0 alignment: 0
Matching edge for corner ('W', 'B', 'R') DBR 0: ('R', 'B') FL 1
Corner ('W', 'B', 'R') DBR 0 is in the D layer
Corner ('W', 'B', 'R') D

In [6]:
white_corners = cube.corners(color_filter=["W"])
corner = None
for corner in white_corners:
    if corner.color == ('W', 'O', 'B'):
        print(f"Found corner {corner}")
        break
print(f"Corner {corner} alignment: {corner.alignment()}")
edge = get_matching_edge(corner)
print(f"Matching edge for corner {corner}: {edge}")
f2l_place_corner_edge(cube, corner, edge)
print_color_cube(cube)


Found corner ('W', 'O', 'B') DFL 2
Corner ('W', 'O', 'B') DFL 2 alignment: 1
Matching edge for corner ('W', 'O', 'B') DFL 2: ('O', 'B') FL 1
Corner ('W', 'O', 'B') DFL 2 is in the D layer
Corner ('W', 'O', 'B') DFL 2 is not in the DFR position, rotating cube
Corner ('W', 'O', 'B') DBL 2 is not in the DFR position, rotating cube
Corner ('W', 'O', 'B') DBR 2 is not in the DFR position, rotating cube
Corner ('W', 'O', 'B') DFR 2 is now in the DFR position
Corner ('W', 'O', 'B') DFR 2, alignment: 1 is in the wrong position

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [7]:
print_color_cube(cube)



K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [8]:
def is_f2l_solved(cube):
    """
    Check if F2L is solved
    """
    # check bottom layer
    for position in range(9):
        if cube.get_sticker('D', position) != cube.face_color('D'):
            return False
    # check R, F , L, B faces
    for face in ['R', 'F', 'L', 'B']:
        for position in range(3, 9):
            if cube.get_sticker(face, position) != cube.face_color(face):
                return False
    return True


In [9]:
cube.scramble(20)
solver.cross()
solver.invert_cube()
white_corners = cube.corners(color_filter=["W"])
counter = 0
while not is_f2l_solved(cube):
    counter += 1
    if counter > 100:
        print("F2L not solved after 100 iterations, breaking out of the loop")
        break
    for corner in white_corners:
        edge = get_matching_edge(corner)
        sentinel = 0
        success = False
        while not success and sentinel < 10:
            print(f"Attempt {sentinel}: Trying to place corner {corner} and edge {edge}")
            success = f2l_place_corner_edge(cube, corner, edge)
            sentinel += 1
            for c in white_corners:
                print(f"  Corner {c} alignment: {c.alignment()}")

Attempt 0: Trying to place corner ('W', 'R', 'G') UBL 1 and edge ('R', 'G') FL 0
Corner pre_alignment: ('W', 'R', 'G') UBL 1, 2
Corner post_alignment: ('W', 'R', 'G') UBL 1, 2
Corner ('W', 'R', 'G') UFR 1 is now in the UFR position
edge.position in edge_sequences: BR True
Edge ('R', 'G') UR 0 
Corner condition: true, Edge condition: true
U true UR true sequence: U,R,U',R'
Executing sequence for corner ('W', 'R', 'G') UBR 1 and edge ('R', 'G') UR 0: U,R,U',R'
  Corner ('W', 'R', 'G') UBL 0 alignment: 0
  Corner ('W', 'B', 'R') DBL 0 alignment: 2
  Corner ('W', 'O', 'B') UBR 2 alignment: 1
  Corner ('W', 'G', 'O') UFR 1 alignment: 1
Attempt 1: Trying to place corner ('W', 'R', 'G') UBL 0 and edge ('R', 'G') FR 0
Corner pre_alignment: ('W', 'R', 'G') UBL 0, 0
Corner U: ('W', 'R', 'G') UBR 0, 1
Corner U: ('W', 'R', 'G') UFR 0, 2
Corner post_alignment: ('W', 'R', 'G') UFR 0, 2
Corner ('W', 'R', 'G') UFR 0 is now in the UFR position
edge.position in edge_sequences: FR False
Edge ('R', 'G') F

In [10]:
# white_corners = cube.corners(color_filter=["W"])
# corner = white_corners[1]
# edge = get_matching_edge(corner)
# sentinel = 0
# success = False
# while not success and sentinel < 10:
#     print(f"Attempt {sentinel}: Trying to place corner {corner} and edge {edge}")
#     success = f2l_place_corner_edge(cube, corner, edge)
#     sentinel += 1
# for corner in white_corners:
#     print(corner)

In [11]:
print_color_cube(cube)


K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [12]:
for position in range(3, 9):
    print(position)

3
4
5
6
7
8


In [13]:
cube.load(state="YOWWWGBRGRBGYYBOOBGGYRGYGBBYYBBBYRRROGOORWYOWOGRROWWWW")



In [14]:
solver.cross()
print_color_cube(cube)


K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [15]:
solver.cross()
print_color_cube(cube)



K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [16]:
cube.load(state="WWWWWWWWWYYYGYORYGGGGGGBYYGBBBGBBOYYRRRRRRRRBOOOOOOOBB")
print_color_cube(cube)


K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  



In [17]:
white_corners = cube.corners(color_filter=["W"])
for corner in white_corners:
    print(f"Corner {corner} alignment: {corner.alignment()}")
    edge = get_matching_edge(corner)
    print(f"Matching edge for corner {corner}: {edge}")
    # f2l_place_corner_edge(cube, corner, edge)

Corner ('W', 'R', 'G') UFL 0 alignment: 3
Matching edge for corner ('W', 'R', 'G') UFL 0: ('R', 'G') FR 0
Corner ('W', 'B', 'R') UFR 0 alignment: 3
Matching edge for corner ('W', 'B', 'R') UFR 0: ('R', 'B') FL 0
Corner ('W', 'O', 'B') UBR 0 alignment: 3
Matching edge for corner ('W', 'O', 'B') UBR 0: ('O', 'B') BR 0
Corner ('W', 'G', 'O') UBL 0 alignment: 3
Matching edge for corner ('W', 'G', 'O') UBL 0: ('O', 'G') BL 0


In [18]:
solver.invert_cube()
corner = white_corners[3]
edge = get_matching_edge(corner)
cube.rotate_cube(axis='Y', clockwise=True)
print(f"Corner {corner} alignment: {corner.alignment()}")
print(f"Matching edge for corner {corner}: {edge}")


Corner ('W', 'G', 'O') DBL 2 alignment: 3
Matching edge for corner ('W', 'G', 'O') DBL 2: ('O', 'G') BL 1


In [19]:
solver.f2l()
print_color_cube(cube)


K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

                                        

                                        

                                        

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

K0 K0 K0            K0 K0 K0  K0 K0 K0  

